## Compute API key
A client API key is the base64 encoded form of a JSON object with the following structure, concatenated using a dot with the SHA256 signature of the same object in compact JSON form:
```
{"name":"Agro Novae Industrie, SAS","id":1}
```
The JSON object can be more complete and in general follow the schema :
```
components-openapi.json#/components/schemas/Client
```

The API key is currently passed in a header but is URL safe, and could be passed as a query parameter instead.

In [1]:
client_name = "Agro Novae Industrie, SAS"
client_id = 1
expires_in = dict(days=30)

In [2]:
# Global settings

# Filename for the salts
filename = '../core/core-implementation/salts.ndjson'



### Update the secret salt
The salts have a limited validity, 9 weeks, which is longer by design than the 30 days for individual API keys. 

The purpose of having a secret, that is included in the signature but not available to the owner to the API key, is to make it difficult to forge API keys. The signature algorithm is easy to guess and without a secret, a rigue user could just forge the client data and directly compute the signature.

In [6]:
# Create and manage a secret salt. It must be persisted to validate received tokens
import json
from datetime import datetime, timedelta

# Compute a secret server salt that changes too, but only once a year
# The purpose of the salt is to make it impossible for a rogue client to compute the correct signature
# after having falsified the client data. The random part makes it hard to guess (the other fields are
# trivial to guess if the structure of the salt leaks out).
current_timestamp = int(datetime.now().timestamp())

# The expiration date of a salt is intended to purge the database.
# A salt made now expires approximately at the end of the next month (validity: at last 1 month)
current_datetime = datetime.now()
expire_datetime = datetime(current_datetime.year,current_datetime.month,1) + timedelta(weeks=9) 
salt = json.dumps({
    "tool":"co2track",
    "exp": int(expire_datetime.timestamp()),
    "random": "b5740a10-244a-4869-bda2-8f8793355103"
}, separators=(',', ':'))
print(f"salt: '{salt}'")


try:
    # Read existing salts to avoid duplicates
    existing_salts = set()
    with open(filename, 'r') as file:
        for line in file:
            content = line.strip()
            if content:
                existing_salts.add(content)

    # Check expiration date
    initial_count = len(existing_salts)
    active_salts = [salt for salt in existing_salts if json.loads(salt)['exp'] > current_timestamp]
    final_count = len(active_salts)
    print(f"Removed {initial_count - final_count} expired salts")
    
    with open(filename, 'w') as file:
        # Write back still valid salts
        for old_salt in active_salts:
            file.write(old_salt + '\n')

        # Write the salt if it's not a duplicate
        if salt not in existing_salts:
            file.write(salt + '\n')
            print("Salt added to file.")
        else:
            print("Duplicate salt. Not added.")

except FileNotFoundError:
    # If file does not exist, create it and add the salt
    with open(filename, 'w') as file:
        file.write(salt + '\n')
    print("File created and salt added.")


salt: '{"tool":"co2track","exp":1722636000,"random":"b5740a10-244a-4869-bda2-8f8793355103"}'
Removed 0 expired salts
Duplicate salt. Not added.


### Compute the Client API key

In [4]:
import json
import hashlib
import base64
from datetime import datetime, timedelta

# Assuming client_name and client_id are defined elsewhere in your script
client = dict(name=client_name, id=client_id)

# Add an expiration timestamp
expiration_date = datetime.now() + timedelta(**expires_in)
client.update(exp=int(expiration_date.timestamp()))
              
# Correct function to convert dictionary to JSON string
json_string = json.dumps(client, separators=(',', ':'))

# Calculate SHA256 hash of the JSON string including a secret server salt
assert salt, "Run the previous cells to define 'salt'"
signed_data = f"{json_string}\n{salt}"
# print(f"Signed data : \n{signed_data}\n")
hash_object = hashlib.sha256(signed_data.encode())
hex_dig = hash_object.hexdigest()

# print("JSON String:", json_string)
# print("SHA256 Hash:", hex_dig)

# Encode JSON string and hash in base64
encoded_json = base64.urlsafe_b64encode(json_string.encode()).decode('utf-8')
encoded_hash = base64.urlsafe_b64encode(hex_dig.encode()).decode('utf-8')

# Join them with a dot, similar to JWT format
jwt_like_token = f"{encoded_json.rstrip('=')}.{encoded_hash.rstrip('=')}"
print(f"Client API key is '{jwt_like_token}'")
print("You must regenerate the server using 'docker-compose up -d --build' to install the new salt.")

Client API key is 'eyJuYW1lIjoiQWdybyBOb3ZhZSBJbmR1c3RyaWUsIFNBUyIsImlkIjoxLCJleHAiOjE3MjA4NzA4NDR9.ZjhjNzIwMTMxMjMwMGJiNGM0MjQzNmE5MTU0OTBkOTg4MTRhN2QzODA3YzY3ZTg0YmNmYjBjMWEwYTJlOTdhYw'
You must regenerate the server using 'docker-compose up -d --build' to install the new salt.


## Validate and decode an API key
Ensure the key is in variable ``jwt_like_token``.
The first step is to split the two parts and decode the base64 encoding.

Next the validator loops over non-expired salts, tries each one in turn and 

In [5]:
import base64
import hashlib
import json
from datetime import datetime

parts = jwt_like_token.split('.')

# Decode each part from Base64
decoded_parts = [base64.urlsafe_b64decode((part + '==').encode()).decode('utf-8') for part in parts]
client = decoded_parts[0]
signature = decoded_parts[1]

print("client:", client)
print("Signature:", signature)

try:
    with open(filename, 'r') as file:
        for line in file:
            # Compute the SHA256 signature
            signed_data = f"{client}\n{line.strip()}"
            # print(f"Signed data attempt:\n{signed_data}\n")
            signature_check = hashlib.sha256(signed_data.encode()).hexdigest()
            
            if signature_check == signature:
                # If a matching signature is found
                client_data = json.loads(client)
                # Check if the client data is not expired
                if client_data['exp'] > int(datetime.now().timestamp()):
                    tool = json.loads(line.strip())['tool']
                    print("Valid API key for client", client_data['name'], "and tool", tool + ".")
                    print("API key valid until", datetime.fromtimestamp(client_data['exp']).strftime('%Y-%m-%d %H:%M:%S'))
                else:
                    print("Client API key is expired.")
                break
        else:
            # Normal loop end: not found
            print("No matching salt found, or the token is invalid.")
except FileNotFoundError:
    print("Salts file does not exist, token cannot be validated.")

client: {"name":"Agro Novae Industrie, SAS","id":1,"exp":1720870844}
Signature: f8c7201312300bb4c42436a915490d98814a7d3807c67e84bcfb0c1a0a2e97ac
Valid API key for client Agro Novae Industrie, SAS and tool co2track.
API key valid until 2024-07-13 13:40:44
